In [ ]:
# Imports
import pandas as pd
import numpy as np
import math
import regex as re
# for nlp tags
import spacy

#prevents warning
import warnings
warnings.filterwarnings('ignore')

# my functions
import pscCatTypeComp as my
which_sh = my.which_sh
get_pos_tags = my.get_pos_tags
assign_pos = my.assign_pos
choose_sh = my.choose_sh
consecutive = my.consecutive
most_frequent = my.most_frequent
jaccard_similarity = my.jaccard_similarity
squared_sum = my.squared_sum
cos_similarity = my.cos_similarity
similarity_scoring = my.similarity_scoring
psc_to_sh = my.psc_to_sh
get_per_vals = my.get_per_vals
search_sequence_numpy = my.search_sequence_numpy
get_sh_from_shs = my.get_sh_from_shs
is_psc = my.is_psc
check_act = my.check_act
get_controls = my.get_controls
remove_space_list = my.remove_space_list
check_old_names = my.check_old_names
format_chips_op = my.format_chips_op
merge_names_nat = my.merge_names_nat
how_different = my.how_different
entity_extract = my.entity_extract
preprocess_df = my.preprocess_df
tag_multiple = my.tag_multiple


nlp = spacy.load("en_core_web_trf")

In [ ]:
df = pd.read_csv("Webservice backlog feb oct.csv")

# ACTION CODE LOAD
df_act = pd.read_csv("Action Code.csv",encoding = 'utf-8')
df_act['ACTION_CODE_EFF_DATE'] = pd.to_datetime(df_act.ACTION_CODE_EFF_DATE)
df_act = df_act.sort_values(by = 'ACTION_CODE_EFF_DATE',ascending = False)

# NATIONALITY LOAD & PREPROCESS
df_off = pd.read_csv("Officer Details.csv")
nationalities = list(df_off.OFFICER_NATIONALITY.apply(lambda x: x.lower() if type(x) == str else '').unique())
nationalities.remove('')
nationalities = [nat for nat in nationalities if ',' not in nat]


df_nat_and_adj = pd.read_csv("Countries and Nationalities.csv")

# making all lower case
for col in df_nat_and_adj.columns:
    df_nat_and_adj[col] = df_nat_and_adj[col].apply(lambda x: x.lower())
    
nationalities = nationalities + [nat for nat in df_nat_and_adj.country.values if nat not in nationalities] + [nat for nat in df_nat_and_adj.adjectivals.values if nat not in nationalities]    
nationalities = list(set(nationalities))


In [ ]:
def preprocess_officers(df):
    df['LB'] = df.DESCRIPTION.apply(lambda x: min([9999] + [int(item) for item in x.replace('%','').split(' ') if item.isdigit() == True]))
    df['UB'] = df.DESCRIPTION.apply(lambda x: max([0] + [int(item) for item in x.replace('%','').split(' ') if item.isdigit() == True]))
    df.loc[df.LB == 9999,'LB'] = np.nan
    df.loc[df.UB == 0,'UB'] = np.nan
    df.loc[(df.LB == 75) & (df.UB == 75),'UB'] = 100
    df['TYPE'] = df.DESCRIPTION.apply(lambda x: 'NOC' if 'shares' in x else 'VR' if 'voting' in x else 'SIC')
    df['AS'] = df.DESCRIPTION.apply(lambda x: 'trust' if 'trust' in x else 'person' if 'person' in x else 'firm')
    
    df['merged_name'] =  [merge_names_nat(x).replace('  ',' ') for x in df[['OFFICER_FORENAME_1','OFFICER_FORENAME_2','OFFICER_SURNAME']].values]
    
    return df

df_off = preprocess_officers(df_off)
df_off.sample(5).head()

# NAME

In [ ]:
def name_extraction_get_df(psc_id,df_name,trace = False):
    # first check action code...
    cont_id = df_name[df_name.PSC_DISCREPANCY_ID == psc_id].CONTACT_ID.values[0]
    
    # fetching data we need to complete
    row_df = df_name[df_name.PSC_DISCREPANCY_ID == psc_id]
    txt = row_df.discrepancy_detail.values[0]
    corp_id = row_df.CORPORATE_BODY_ID.values[0]
    psc_name = row_df.extract_psc_name.values[0].lower()
    
    df_cont = df_off[df_off.CONTACT_ID == cont_id]
    df_cont['merged_name'] = [merge_names_nat(x).replace('  ',' ') for x in df_cont[['OFFICER_FORENAME_1','OFFICER_FORENAME_2','OFFICER_SURNAME']].values]

    sh_names = df_cont.merged_name.unique() #list(set(df_sh_names[df_sh_names['corp_id'] == corp_id].sh_names.values[0]))
    sh_surnames = [name.split(' ')[-1].lower() for name in sh_names if type(name) == str ]

    # using officer information to retrieve sh names
    #off_names = 
    #print("OFFICERS : ",off_names)
    
    # now we have collected all our information we should set up the breakdown dataframe:
    
    # 1) Preprocessing...
    txt_orig = txt
    txt = txt.lower()
    
    remove_punc = ['.','/','? ','[',']',')','(','!','"','£','$','%','^','&','*','\n','\r',',',';',"'s "] # ,' - '
    remove_words = [' and ',' the ',' an ',' for ',' was ',' is ',' as ',' to ',' from ',' it ',' be ',' of ',' part ',' psc ',
                    ' name ',' date ',' her ',' he ',' his ',' hers ',' their ',' theirs ',' him ',' she ',' this ',' them ',
                    ' psc ',' director ',' they ',' there ',' individual ',' dob ','vs ',' ch ',' id ',' a ',' on ',' or ',
                   ' in ']
                   
    txt = txt.replace(':',' : ')
    # removals of chars we dont want:
    for punc in remove_punc:
        txt = txt.replace(punc,' ')
    for word in remove_words:
        txt = txt.replace(word,' ')
        txt = txt.replace(word[:-1]+'-',' ') # replaces things like "name-" rather than just " name "
        
    txt = txt.replace('-',' - ') # for doubel barrel surnames so we can match easier, these will be rejoined later on
    txt = txt.replace("' ",' ').replace(" '",' ').replace('?','')

    # splitting and removal of blank chars
    txt_split = txt.split(' ')
    
    while '' in txt_split:
        txt_split.remove('')
    
    # o/p for testing phase
    if trace == True:
        print("==========================================================================================================")
        print("PSC_ID : "+str(psc_id))
        print("CORP ID : "+str(corp_id))
        print("CONT ID :"+str(cont_id))
        print("PSC Name Given : "+psc_name)
        print("Shareholders : ")
        print(sh_names)
        print("\n<-----The text----->\n"+txt_orig)
    
    
    # 2) Split into df then tagging
    
    # tagging set up
    first_tags = ['1st','forename','first','middle','title']
    second_tags = ['2nd','surname','last','double','barrel','married','maiden']
    title_tags = ['dr','mr','mrs','ms','mz']
    negatives = ['isnt','not','remove','wrong']
    
    # checking if talking about married
    married_markers = ['married','marriage','maiden']
    married_test = False
    for marker in married_markers:
        if marker in txt_split:
            married_test = True
            
            
    # getting tags
    tags = ['First' if x in first_tags else 'Second' if x in second_tags else 'Negative' if x in negatives else 'Title' if x in title_tags else 'JOIN' if x == '-' else 'JOIN_SPECIAL' if x in [':',';'] else np.nan for x in txt_split]
    
    # getting counts of first and second tag occurances...
    first_count = tags.count('First')
    second_count = tags.count('Second')
    
    
    # checking if any obvious sh matches in txt
    associated_sh = [which_sh(x,sh_names,psc_name) for x in txt_split]
    
    # creating df
    df_txt = pd.DataFrame({'word':txt_split,'tags':tags,'associated_sh':associated_sh})
    
    # we also want to add spacy name ent tags to identify areas where name might mismatch...
    pos_tags = get_pos_tags(txt_orig,nlp,reduce_ents = True)
    pos_in_txt = [assign_pos(pos_tags,x) for x in txt_split]
    df_txt['pos'] = pos_in_txt
    df_txt['pos'][~df_txt.associated_sh.isna()] = 'PERSON'
    
    df_txt.loc[df_txt.pos.isna(),'pos'] = df_txt.loc[df_txt.pos.isna(),'word'].apply(lambda x: 'PERSON' if (x in ' '.join(sh_names) and x != '-') else np.nan)
    
    # we want to check the JOIN and see if names are either side for double barrels
    inds_join = df_txt[df_txt.tags == 'JOIN'].index
    join_names_inds = []
    for ind in inds_join:
        lower = ind-1 if ind-1 >= 0 else ind
        
        surrounding = df_txt.pos.iloc[[ind-1,ind,ind+1]] if ind-1 >= 0 else df_txt.pos.iloc[[ind,ind+1]] # gets surrounding pos tags
        if 'PERSON' in surrounding.values: # checks if person is being spoken about
            if ind -1 >= 0:
                df_txt.loc[[ind-1,ind,ind+1],'pos'] = 'PERSON' # assigns person tag to join
            else:
                df_txt.loc[[ind,ind+1],'pos'] = 'PERSON'
            join_names_inds.append(ind)
     
    # below checked for tag situation of "PERSON nan PERSON" and we assume the sandwiched nan is also PERSON or part-of-name
    inds_to_tag = []
    for ind in df_txt.index[2:]:
        if (df_txt.loc[ind,'pos'] == 'PERSON') and (df_txt.loc[ind-2,'pos'] == 'PERSON'):
            inds_to_tag.append(ind-1)
    
    df_txt.loc[inds_to_tag,'pos'] = 'PERSON'
    
    # creating matched column
    df_txt['matched'] = False
    df_txt['matched'][(~df_txt.associated_sh.isna()) & (df_txt.pos == 'PERSON') ] = True
    df_txt['matched'][df_txt.tags == 'JOIN'] = True
    
    if df_txt.shape[0] >=2:
        if df_txt.pos.values[-2] == 'PERSON':
            df_txt.loc[df_txt.shape[0]-1,'pos'] = 'PERSON'
        
    df_txt.loc[df_txt[df_txt.tags == 'Negative'].index,'pos'] = np.nan    
    
    # searching for surname: or surname - tags, same for forename
    drop_point = []
    for i in range(0,df_txt.shape[0]):
        tag = df_txt.loc[i,'tags']
        if (tag in ['First','Second']) and ( i < df_txt.shape[0]-1):
            if (df_txt.loc[i+1,'tags'] == 'JOIN_SPECIAL'):
                df_txt.loc[i+2,'pos'] = 'PERSON'
                drop_point.append(i+1)
                drop_point.append(i)
    
    df_txt = df_txt.drop(drop_point)
    df_txt.reset_index(drop = True,inplace = True)
    
    df_txt['associated_sh'] = df_txt['associated_sh'].apply(lambda x: choose_sh(x,psc_name))
    
    # additional name capture element
    person_inds = list(df_txt[df_txt.pos == 'PERSON'].index)
    for ind in person_inds:
        current = df_txt.loc[ind,'word']
        current = current[0].upper() + current[1:]
        
        if ind-1 >= df_txt.index[0]:
            consider_before = df_txt.loc[ind-1,'word']
            
            if ((consider_before[0] < 'a' or consider_before[0] > 'z')):
                continue
                
            consider_before = consider_before[0].upper() + consider_before[1:]
            
            if ' '.join([consider_before,current]) in txt_orig:
                df_txt.loc[ind-1,'pos'] = 'PERSON'
            
        if ind+1 <= df_txt.index[-1]:
            consider_after = df_txt.loc[ind+1,'word']
            
            if ((consider_after[0] < 'a' or consider_after[0] > 'z')):
                continue
            
            consider_after = consider_after[0].upper() + consider_after[1:]
            
            if current + ' ' +consider_after in txt_orig:
                df_txt.loc[ind+1,'pos'] = 'PERSON'
        
    
    
    # need a final check for consecutive sh where difference in names, we will take first as assumed subject
    inds_peoples = df_txt[~df_txt.associated_sh.isna()].index
    if len(inds_peoples) > 1:
        prev_ind = inds_peoples[0]
        for i in range(1,len(inds_peoples)):
            ind = inds_peoples[i]
            if ind-1 == prev_ind:
                df_txt.loc[ind,'associated_sh'] = df_txt.loc[ind-1,'associated_sh']

            prev_ind = ind
            
    noc_buzzwords = ['nature of control','ownership','shares','voting']           
    noc_found = any([True if x in txt else False for x in noc_buzzwords]) == True

    return df_txt, psc_name, sh_names, married_test, [first_count,second_count],noc_found

In [ ]:
def type_of_issue(psc_id,df_txt,psc_name,sh_names,married,counts,noc_found,trace = False):
    
    cont_id = df[df.PSC_DISCREPANCY_ID == psc_id].CONTACT_ID.values[0]
    
    txt_split = df_txt['word'].values.tolist()
    sh_surnames = [name.split(' ')[-1].lower() for name in sh_names if type(name) == str ]
    
    # extract the areas where PERSON sequence is in pos but not all matched
    inds_person = list(df_txt[df_txt.pos == 'PERSON'].index)
    inds_groups = consecutive(inds_person)
    if len(inds_person) > 0:
        for i in range(0,len(inds_groups)):
            group = inds_groups[i]
            
            # checking group hasnt captured two consecutive
            group_name = list(df_txt.loc[group,'word'].values)
            first_name = group_name[0]
            if first_name in group_name[1:]: # checks if name is later part of group
                ind_to_split = group_name[1:].index(first_name) + 1
                first_group = group[:ind_to_split]
                second_group = group[ind_to_split:]

                del inds_groups[i]
                inds_groups.append(first_group)
                inds_groups.append(second_group)
        
    # going through groups and only  keep those where not all are matched
    unmatched_groups = [] 

    for group in inds_groups:
        current_name = ' '.join(df_txt.loc[group,'word']).replace(' - ','-')

        if (current_name != psc_name) and (current_name not in sh_names) : # checks if all matched
            unmatched_groups.append(group)
     
    # want to see groups so will convert to text
    group_words = []
    for group in inds_groups:
        name = ' '.join(df_txt.loc[group,'word']).replace(' - ','-')
        group_words.append(name)
        
    while '' in group_words:
        group_words.remove('')
    
    # we want to remove titles from groups
    titles = ['dr','mr','mrs','mx','ms','miss','master','sir','lord']
    for i in range(len(group_words)):
        group = group_words[i]
        for item in titles:
            if item in group.split(' '):
                group_split = group.split(' ')
                group_split.remove(item)
                group_words[i] = ' '.join(group_split)
            
    # When no groups are found :
    if len(group_words) == 0:
        if counts[1] > 0: #surname mentioned
            if married == True: # check if about marriage
                return ['MARRIAGE - LOW',[psc_name],['Error: see discrepancy details']]
            else:
                return ['SURNAME - HIGH',[psc_name],['Error: see discrepancy details']]
            
        elif counts[0] > 0: # forename mentioned surname not
            return ['FORENAME - NONE',[psc_name],['Error: see discrepancy details']]
        
        else: # no mention of either
            if married == True: # check if about marriage
                return ['MARRIAGE - LOW',[psc_name],['Error: see discrepancy details']]
            else:
                return ['COULD NOT DETERMINE - HIGH',[psc_name],['Error: see discrepancy details']]
            
    
    removal_words = ['forename','first name','second name','middle name','surname','maiden','married','should',]
    unmatched_words = []
    for group in unmatched_groups:
        name = ' '.join(df_txt.loc[group,'word']).replace(' - ','-')
        for rem in removal_words:
            name = name.replace(rem,'')
        unmatched_words.append(name)
    
    
    unmatched_words = list(set(unmatched_words))
    #print(unmatched_words)
    # we want to remove titles from groups
    titles = ['dr','mr','mrs','mx','ms','miss','master','sir','lord']
    for i in range(len(unmatched_words)):
        group = unmatched_words[i]
        for item in titles:
            if item in group.split(' '):
                group_split = group.split(' ')
                group_split.remove(item)
                unmatched_words[i] = ' '.join(group_split)
    
    
    # checking if shs_in_txt if multiple officers...
    df_sliced = df_off[df_off.CONTACT_ID == cont_id] # gets relevant officers info
    
    if psc_name != 'psc missing':
        sh_names = psc_to_sh(df_sliced.merged_name.unique(),psc_name)
    
    for name in unmatched_words:
        if name == sh_names:
            print(name)
            unmatched_words.remove(name)

    # removing empties
    while '' in group_words:
        group_words.remove('')
    while '' in unmatched_words:
        unmatched_words.remove('')
    
    
    
    if trace == True:
        print("\nGroups: ")
        print(group_words)
        print("\nUnmatched :")
        print(unmatched_words)
        print("\n")
        
    shs_in_txt = df_txt.associated_sh.values if psc_name == 'psc missing' else [sh_names]
    shs_split = []
    for sh in shs_in_txt:
        if type(sh) == str:
            if '/' in sh:
                for s in sh.split('/'):
                    if s not in shs_split:
                        shs_split.append(s)
                continue
            if sh not in shs_split:
                shs_split.append(sh)
        
    shs_in_txt = shs_split # this will hold all share holders found in text, any undecided are split and both added
    differences = [] # stores difference per unmatched item
    
    officer_names = list(df_sliced.merged_name.unique())
    officer_ids = df_sliced[df_sliced.merged_name == psc_to_sh(officer_names,psc_name)].OFFICER_ID.unique()

    if trace == True:
        print("OFFICER NAMES : ",officer_names)
        print("OFFICER IDS : ",officer_ids)
        print("REMAINING UNMATCHED : ",unmatched_words)
    
    # CHECKS FOR MULTIPLE IDS OR NAMES UNDER SINGLE PERSON
    if len(officer_ids) == 1: # if one found we need to check if many names are listed
        if df_sliced[df_sliced.OFFICER_ID == officer_ids[0]].shape[0] > 1:
            off_names = list(df_sliced[df_sliced.OFFICER_ID == officer_ids[0]].merged_name.unique())
            off_surs = list(set([name.split(' ')[-1] for name in off_names]))
            if len(off_surs) > 1:
                return ['MULTIPLE ENTRIES FOR ID SURNAME - HIGH',off_names,unmatched_words]
            elif len(off_names) > 1:
                return ['MULTIPLE ENTRIES FOR ID FORENAME - LOW',off_names,unmatched_words]

            
    elif len(officer_ids) < len(shs_in_txt):
        off_names = list(df_sliced[df_sliced.OFFICER_ID == officer_ids[0]].merged_name.unique())
        off_surs = list(set([name.split(' ')[-1] for name in off_names]))
        if len(off_surs) > 1:
            return ['MULTIPLE ENTRIES FOR ID SURNAME - HIGH',off_names,unmatched_words]
        elif len(off_names) > 1:
            return ['MULTIPLE ENTRIES FOR ID FORENAME - LOW',off_names,unmatched_words]    
        
    # CHECKING REMAINING UNMATCHED
    if psc_name.lower() == 'psc missing':
        psc_name = most_frequent(shs_in_txt)
    
    our_sh = psc_to_sh(officer_names,psc_name) # gets sh name associated with psc
    our_sur = our_sh.split(' ')[-1]
    our_fore = our_sh.split(' ')[:-1]
    differences = []
    if len(unmatched_words) > 0:
        for unmatched in unmatched_words:
            if unmatched == our_sh.lower():
                continue
            if len(unmatched.split(' ')) == 1: # if only one name in unmatched
                if (unmatched in our_fore) or (unmatched == our_sur):
                    differences.append("ALL MATCHED - LOW")
                elif counts[1] > 0:
                    differences.append("SURNAME - HIGH" if how_different(unmatched,our_sur) > 2 else 'SURNAME - MEDIUM')
                else:
                    differences.append("FORENAME - NONE")
                    
            elif len(unmatched.split(' ')) > 1: # the case where current item is larger than 1
                
                unmatched_sur = unmatched.split(' ')[-1]
                unmatched_fore = unmatched.split(' ')[:-1]
                
                if unmatched_sur != our_sur: # if surname doesnt match
                    differences.append('SURNAME - HIGH' if how_different(unmatched,our_sur) > 2 else 'SURNAME - MEDIUM')
                    
                elif unmatched_fore != our_fore: # if surname does match
                    differences.append('FORENAME - NONE')
                    
                else:
                    continue # if all matched move on
    else:
        differences.append("ALL MATCHED - NONE")
        
    priorities_ranked = ['SURNAME - HIGH','SURNAME - MEDIUM','FORENAME - NONE']
    priority_assigned = 'ALL MATCHED - NONE'
    
    for prio in priorities_ranked: # getting highest priority
        if prio in differences:
            priority_assigned = prio
            break
            
    if noc_found == True: # checking if NOC has been mentioned as this is common and should be HIGH
        return ['NOC MENTIONED - HIGH',shs_in_txt,unmatched_words]
    
    if (priority_assigned in priorities_ranked[:2]) and (married == True): # checks if discrepancy is about marriage only if a higher priority is currently assigned
        priority_assigned = 'MARRIAGE - LOW'
        
    if priority_assigned == 'ALL MATCHED - NONE':
        unmatched_words = group_words
        shs_in_txt = [sh_names] if type(sh_names) != list else sh_names
        
    return [priority_assigned,shs_in_txt,unmatched_words] # returns out priority, shareholders in text, and unmatched items

In [ ]:
def name_prio(psc_id,df):
    print("CURRENT ID : ",psc_id)
    df_txt, psc_name, sh_names, married_test, counts,noc = name_extraction_get_df(psc_id,df,trace = False)
    op = type_of_issue(psc_id,df_txt,psc_name,sh_names,married_test,counts,noc,trace = False)
    # Assigning op to columns
    index = df[df.PSC_DISCREPANCY_ID == psc_id].index[0]
    df.loc[index,'priority_explained'] = op[0].split(' - ')[0] 
    df.loc[index,'priority'] = op[0].split(' - ')[1]
    df.loc[index,'CHIPS_value'] = ' | '.join(op[1])
    df.loc[index,'material_nonmaterial'] = 'MATERIAL' if ' NONE' not in op[0] else 'NON-MATERIAL'
    to_return = ' | '.join(op[2]) 
    return to_return

# NOC

In [ ]:
def text_breakdown(df,psc_id):
    # This function will return a dataframe which contains a breakdown of descrip_detail and tags per word
    # This allows for easier decision making
    
    # => Getting Text and Shareholders <=
    txt = df[df.PSC_DISCREPANCY_ID == psc_id].discrepancy_detail.iloc[0]
    print("TEXT : ",txt)
    comp_num = df[df.PSC_DISCREPANCY_ID == psc_id].COMPANY_NUMBER.iloc[0]
    corp_id = df[df.PSC_DISCREPANCY_ID == psc_id].CORPORATE_BODY_ID.iloc[0]
    psc_name = df[df.PSC_DISCREPANCY_ID == psc_id].extract_psc_name.iloc[0]
    
    df_sh = df_off[df_off.CORPORATE_BODY_ID == corp_id]
    df_sh['merged_name'] = [merge_names_nat(x).replace('  ',' ') for x in df_sh[['OFFICER_FORENAME_1','OFFICER_FORENAME_2','OFFICER_SURNAME']].values]
    shs = df_sh['merged_name'].unique()

    # => PREPROCESSING AND POS TAGGING <=
    pos_tags = get_pos_tags(txt,nlp,reduce_ents = True)
    
    # preprocessing
    txt = txt.lower()
    
    if '  ' in txt:
        txt.replace('  ',' ')
    
    
    stop_words = ['.','!','\n','\r','>','<'] # these will be changed to have a space either side
    extra_chars = [',','?','"',':',';','£','$','^','&','*','(',')','_','/','-',"'",' fca '] # these are removed
    
    for stop in stop_words:
        txt = txt.replace(stop, ' '+stop+' ') # replacing stop words
    for ext in extra_chars:
        txt = txt.replace(ext,' ') # removing extra chars
    
    txt = txt.replace('%','% ')
    txt_split = txt.split(' ') # splits text word by word

    while '' in txt_split:
        txt_split.remove('') # removes extra blank spaces in split as we do not care for these
        

    # creating tags for each type...
    shares = ['shares','share','sh','class','shareholding','shareholder','holding','owns','owner','holds','own','shared','shareholdings']
    voting = ['vote','voting','rights']
    sic = ['right','sic','significant','noc','appoint','directors','influence','control']
    
    type_tag = []
    for word in txt_split:
        if word in stop_words:
            type_tag.append('STOP')
            continue
        elif word in shares:
            type_tag.append('OWNERSHIP')
            continue
        elif word in sic:
            type_tag.append('SIC')
            continue
        elif word in voting:
            type_tag.append('VOTING')
            continue
        elif word in ['not','remove','false','wrong','incorrect','no','removed']:
            type_tag.append('NEGATIVE')
            continue
        elif word in ['more','greater','>','over']:
            type_tag.append('GREATER')
            continue 
        elif word in ['less','<','under']:
            type_tag.append('LESSER')
            continue 
        elif (word == 'and') or (word == '&'):
            type_tag.append('JOIN')
            continue
        elif word == 'ownership':
            type_tag.append('SH/VOTE')
            continue 
        type_tag.append(np.nan)
    # ---> end of loop <---
    
    df_res = pd.DataFrame(columns = ['word','pos','type_tag','sh_name','per_value']) # sets up columns in df
    # we need to ensure non key words aren't tagged so we add a boolean array to apply to
    ignor_common = ['on','the','in','of','and','companies','house','than']
    ignor_bool = [True if word not in ignor_common else False for word in txt_split]
    
    
    df_res['word'] = txt_split # assigns text split to words
    df_res['type_tag'] = type_tag
    df_res['pos'][ignor_bool] = df_res.word[ignor_bool].apply(lambda x: assign_pos(pos_tags,x)) # gets tags per word
    df_res['sh_name'][ignor_bool] = df_res.word[ignor_bool].apply(lambda x: which_sh(x,shs,psc_name))
    df_res['sh_name'] = df_res.sh_name.fillna(value = np.nan)
    df_res['per_value'] = df_res.word.apply(get_per_vals)

    # the percentage column is taking 70.50% as 70.0 and 50.0%, we need to have the intelligence to 
    inds_per_val = df_res[~df_res.per_value.isna()].index
    for i in range(1,len(inds_per_val)):
        current_ind = inds_per_val[i]
        prev_ind = inds_per_val[i-1]
        if prev_ind == current_ind-2:
            if df_res.word.loc[current_ind - 1] == '.':
                combined_per = str(df_res.per_value.loc[prev_ind]).split('.')[0]+'.'+str(df_res.per_value.loc[current_ind]).split('.')[0]
                df_res.per_value.loc[prev_ind] = np.nan
                df_res.per_value.loc[current_ind] = float(combined_per)
    
    # when it is indicisive of who the name belongs to we can use surrounding names to know
    # first we need th indicies where two or more names have been linked
    non_nan_names = df_res[~df_res.sh_name.isna()]
    bool_multi_names = [True if '/' in x else False for x in non_nan_names.sh_name.values]
    inds_multi_names = non_nan_names[bool_multi_names].index # gets indices where mutliple are found
    
    for ind in inds_multi_names:
        surrounding_names = df_res.sh_name.iloc[ind-1:ind+1].values
        all_names = []
        for item in surrounding_names: # if the name could not be determined we split and consider both
            if type(item) == str:
                if '/' in item:
                    item_split = item.split('/')
                    for part in item_split:
                        all_names.append(part)
                else:
                    all_names.append(item) # if name was decded then append
        
        # now we want to find the most common
        all_name_counts = [0 for i in range(len(all_names))] # found counts to be appended to
        
        for name in all_names: # goes through all found names incl. duplicates for count purposes
            all_name_counts[int(all_names.index(name))] += 1 # adds 1 to count of name where name is found
        
        df_res.sh_name.iloc[ind] = all_names[all_name_counts == max(all_name_counts)] # appends data to have decided best name 
    
    # merging pos and type_tag
    df_res.type_tag[df_res.type_tag.isna()] = df_res[df_res.type_tag.isna()].pos
    df_res.type_tag[~df_res.per_value.isna()] = 'DIGIT'
    df_res.type_tag[~df_res.sh_name.isna()] = df_res[~df_res.sh_name.isna()].pos
    
    # assigning NAME tag to location with name match
    #df_res.pos[~df_res.sh_name.isna()] = 'NAME'
    
    df_res.type_tag[(df_res.type_tag.isna()) & (~df_res.sh_name.isna())] = 'PERSON'
    
    df_res = df_res[['word','type_tag','sh_name','per_value']]
    df_res['is_digit'] = df_res.type_tag.apply(lambda x: True if type(x) != str else False)
    
    df_res = df_res[~df_res.type_tag.isna()]
    df_res.sh_name[df_res.sh_name == 'PERSON'] = str(psc_name).lower()
    df_res = df_res.reset_index(drop = True)
    
    # checking dates havent been captured as digit
    inds_digit = df_res[df_res.type_tag == 'DIGIT'].index.values
    for i in range(0,len(inds_digit)):
        ind = inds_digit[i]
        if i >= 1:
            if (df_res.loc[ind-1,'type_tag'] == 'DATE') or (df_res.loc[ind+1,'type_tag'] == 'DATE'):
                df_res.loc[ind,'type_tag'] = 'DATE'
                df_res.loc[ind,'per_value'] = np.nan
        else:
            if (df_res.loc[ind+1,'type_tag'] == 'DATE'):
                df_res.loc[ind,'type_tag'] = 'DATE'
                df_res.loc[ind,'per_value'] = np.nan
    
    # dealing with names that aren't full captured
    inds_found = df_res[~df_res.sh_name.isna()].index.values
    
    if len(inds_found) > 0:
        for ind in inds_found:
            # forward propagation
            current_tag = 'PERSON'
            ind_new = ind + 1
            while (current_tag == 'PERSON') and (ind_new < df_res.shape[0]):
                current_tag = df_res.loc[ind_new,'type_tag']
                if  current_tag == 'PERSON':
                    df_res.loc[ind_new,'sh_name'] = df_res.loc[ind,'sh_name']
                
                ind_new += 1
            
            # backward propagation
            current_tag = 'PERSON'
            ind_new = ind -1
            while (current_tag == 'PERSON') and (ind_new >= 0):
                current_tag = df_res.loc[ind_new,'type_tag']
                if current_tag == 'PERSON':
                    df_res.loc[ind_new,'sh_name'] = df_res.loc[ind,'sh_name']
                
                ind_new -= 1
                
    # finally we want to get names into the sh_name column where an sh has not been identified but its a different individual
    ind_ppl = df_res[(df_res.type_tag == 'PERSON') & (df_res.sh_name.isna())].index.values
    
    if len(ind_ppl) > 1:
        groups_ppl = []
        current_group = [ind_ppl[0]]

        for i in range(1,len(ind_ppl)): # creating groups
            if ind_ppl[i]-1 == ind_ppl[i-1]:
                current_group.append(ind_ppl[i])
            else:
                groups_ppl.append(current_group)
                current_group = [ind_ppl[i]]

        groups_ppl.append(current_group)

        for group in groups_ppl: # setting names
            name = ' '.join(df_res.loc[group,'word'].values)
            df_res.loc[group,'sh_name'] = name

    elif len(ind_ppl) == 1:
        df_res.loc[ind_ppl,'sh_name'] = df_res.loc[ind_ppl,'sh_name']
    
    df_res.loc[~df_res.sh_name.isna(),'sh_name']= df_res.loc[~df_res.sh_name.isna(),'sh_name'].apply(lambda x:is_psc(x,psc_name))
    
    # removal of psc check
    removal_psc_tags = [
        'removed from the psc','remove from the psc','remove psc','no longer psc','no longer a psc','psc to be removed', 'remove psc'
    ]
    remove = False
    for tag in removal_psc_tags:
        if tag in txt:
            remove = True
    #print(txt)
    
    # checking for limited by guarantee
    limited_by = True if ('limited by guarantee' in txt) or ('ltd by guarantee' in txt) or ('no share company' in txt) or ('company has no shares' in txt) else False

    return df_res, psc_name, remove,limited_by

In [ ]:
def breakdown_to_op(psc_id,df_bd,df,psc_name,rem_tag,limited_tag):
    # checking action codees
    cont_id = df[df.PSC_DISCREPANCY_ID == psc_id].CONTACT_ID.iloc[0]
    
    # data fetching
    
    psc_name = psc_name.lower() if type(psc_name) == str else df[df.PSC_DISCREPANCY_ID == psc_id].extracted_name.values[0]
    if psc_name != '':   
        while psc_name[0] == ' ':
            psc_name = psc_name[1:]
    
    # checking if removal of psc
    if rem_tag == True:
        return ['REMOVAL - HIGH',psc_name,psc_name], df_bd
    
    # Extracting SH in text and determining if multiple found
    sh_in_txt = df_bd[~df_bd.sh_name.isna()].sh_name.unique()
    sh_in_txt = [is_psc(x,psc_name) for x in sh_in_txt]
    
    while '' in sh_in_txt:
        sh_in_txt.remove('')
    
    if (psc_name == '') and (len(sh_in_txt) > 1):
        return ['MULTIPLE ISSUE - HIGH',sh_in_txt,sh_in_txt], df_bd
    elif (psc_name == '') and (len(sh_in_txt) == 1):
        psc_name = sh_in_txt[0]
    elif (psc_name == '') and (len(sh_in_txt) == 0):
        return ['INSUFFICIENT INFO - LOW',sh_in_txt,sh_in_txt],df_bd
    
    count_found = len(sh_in_txt)
    count_org = 0
    
    # Fetching extra info thats needed later on
    df_noc_red = df_off[df_off.CONTACT_ID == cont_id]
    df_noc_red['merged_name'] = [merge_names_nat(x).replace('  ',' ') for x in df_noc_red[['OFFICER_FORENAME_1','OFFICER_FORENAME_2','OFFICER_SURNAME']].values]
    corp_id = df[df.PSC_DISCREPANCY_ID == psc_id].CORPORATE_BODY_ID.iloc[0]
    shs = df_noc_red[df_noc_red.CONTACT_ID == cont_id].merged_name.unique()
    shs = remove_space_list(shs)
    sh_in_txt = sh_in_txt if count_found > 0 else [is_psc(get_sh_from_shs(psc_name,shs),psc_name)]
    sh_in_txt = sh_in_txt if sh_in_txt[0] != None else [psc_name.lower()]
    #print("corp: ",corp_id)
    #print("contact: ",cont_id)
    
    # need to confirm singulart organisation is spoken about if psc is org...
    org_parts = df_bd[df_bd.type_tag == 'ORG'].index.values
    orgs = []
    if psc_name.lower() in ' '.join(df_bd.loc[org_parts,'word']): # below is logic for dealing with non identified organisations in text
        org_inds = df_bd[(df_bd.type_tag == 'ORG') & (df_bd.sh_name.isna())]
        if org_inds.shape[0] != 0:
            prev_ind = org_inds.index.values[0] -1
            current_org = []
            for ind in org_inds.index.values:
                if ind - 1 == prev_ind:
                    current_org.append(org_inds.loc[ind,'word'])
                else:
                    orgs.append(' '.join(current_org))
                    current_org = []
                    current_org.append(org_inds.loc[ind,'word'])
                prev_ind = ind
                
            orgs.append(' '.join(current_org))    
            orgs = list(set(orgs))
            orgs = remove_space_list(orgs)
            for org in orgs: # if any of the orgs arent in the shareholders store or arent the psc name then we know multiple                   
                if (psc_name != is_psc(org,psc_name)) or (org not in shs):
                    count_org += 1
                    break
    
    
    
    # Check if only 'psc' or 'pscs' stated in text
    if (count_found == 1) and (sh_in_txt[0] == 'PERSONS'):
        sh_in_txt = [get_sh_from_shs(psc_name,shs)]
        
    if (count_found == 1) and (sh_in_txt[0] == None):
        sh_in_txt = [get_sh_from_shs(psc_name,shs)]
        
    count_found = len(sh_in_txt) + count_org
    
    # Fetching Relevant Information from PSC Register info (officer information)
    df_noc_red = df_noc_red.loc[[df_noc_red[df_noc_red.NATURE_OF_CONTROL_TYPE_ID == x].first_valid_index() for x in df_noc_red.NATURE_OF_CONTROL_TYPE_ID.unique()]]
    
    # Checking if multiple issue
    if count_found > 1:
        return ['MULTIPLE ISSUE - HIGH',sh_in_txt+orgs,shs],df_noc_red
    
    # Defining different key patterns of markers for each phrase
    key_patterns = {'NOC':['SH/VOTE OWNERSHIP JOIN VOTING VOTING GREATER DIGIT NEGATIVE GREATER DIGIT', # ORDER MATTERS
                                      'GREATER DIGIT LESSER DIGIT SH/VOTE OWNERSHIP JOIN VOTING VOTING',
                                      'DIGIT GREATER SH/VOTE OWNERSHIP JOIN VOTING VOTING',
                                      'OWNERSHIP GREATER DIGIT OWNERSHIP JOIN VOTING VOTING',
                                      'VOTING VOTING JOIN OWNERSHIP OWNERSHIP PERCENT DIGIT DIGIT',
                                      'OWNERSHIP JOIN VOTING VOTING GREATER DIGIT',
                                      'OWNERSHIP GREATER DIGIT NEGATIVE GREATER PERCENT DIGIT',
                                      'SH/VOTE OWNERSHIP GREATER DIGIT NEGATIVE GREATER DIGIT',
                                      'OWNERSHIP GREATER DIGIT LESSER DIGIT',
                                      'GREATER DIGIT NEGATIVE GREATER DIGIT OWNERSHIP',
                                      'OWNERSHIP GREATER DIGIT NEGATIVE GREATER DIGIT',
                                      'SIC GREATER DIGIT NEGATIVE GREATER DIGIT',
                                      'OWNERSHIP PERCENT DIGIT JOIN DIGIT',
                                      'OWNERSHIP PERCENT DIGIT DIGIT',
                                      'SH/VOTE OWNERSHIP DIGIT DIGIT',
                                      'OWNERSHIP PERCENT DIGIT DIGIT',
                                      'OWNERSHIP LESSER PERCENT DIGIT JOIN GREATER DIGIT',
                                      'SH/VOTE OWNERSHIP GREATER DIGIT',
                                      'SIC DIGIT GREATER SH/VOTE OWNERSHIP',
                                      'NEGATIVE GREATER DIGIT OWNERSHIP', # the removal case
                                      'OWNERSHIP DIGIT GREATER',
                                      'OWNERSHIP PERSON DIGIT OWNERSHIP',
                                      'SIC GREATER DIGIT OWNERSHIP',
                                      'PERSON DIGIT OWNERSHIP',
                                      'PERSON DIGIT SH/VOTE OWNERSHIP',
                                      'OWNERSHIP DIGIT OWNERSHIP',
                                      'SIC DIGIT GREATER STOP',
                                      'OWNERSHIP DIGIT OWNERSHIP',
                                      #'GREATER DIGIT LESSER DIGIT',
                                      #'GREATER DIGIT NEGATIVE GREATER DIGIT',
                                      'DIGIT GREATER OWNERSHIP',
                                      'DIGIT OWNERSHIP',
                                      'OWNERSHIP PERCENT DIGIT',
                                      'OWNERSHIP DIGIT OWNERSHIP',
                                      'SIC DIGIT OWNERSHIP GREATER',
                                      'SIC GREATER DIGIT',
                                      'OWNERSHIP DIGIT SIC',
                                      'OWNERSHIP DIGIT STOP',
                                      'DIGIT GREATER SH/VOTE OWNERSHIP',
                                      'OWNERSHIP STOP DIGIT STOP',
                                      'DIGIT VOTING VOTING'], 
                   'VR':['VOTING VOTING GREATER DIGIT NEGATIVE GREATER PERCENT DIGIT',
                                  'VOTING VOTING GREATER DIGIT NEGATIVE GREATER DIGIT',
                                  'VOTING VOTING GREATER DIGIT LESSER DIGIT',
                                  'VOTING VOTING PERCENT DIGIT JOIN DIGIT',
                                  'VOTING VOTING PERCENT DIGIT DIGIT',
                                  'VOTING VOTING DIGIT DIGIT',
                                  'VOTING VOTING DIGIT PERCENT DIGIT',
                                  'GREATER DIGIT NEGATIVE GREATER DIGIT VOTING VOTING',
                                  'VOTING VOTING GREATER DIGIT NEGATIVE DIGIT',
                                  'VOTING VOTING LESSER PERCENT DIGIT JOIN GREATER DIGIT',
                                  'VOTING VOTING GREATER DIGIT',
                                   'VOTING VOTING DIGIT GREATER',
                                   'DIGIT OWNERSHIP JOIN VOTING VOTING STOP',
                                    'VOTING VOTING PERCENT DIGIT'],
                   'SIC':['NEGATIVE SIC SIC SIC','SIC SIC JOIN NEGATIVE SIC','SIC SIC NEGATIVE SIC','SIC SIC SIC']}
    
    # please note the appove has been created by successive runs on various psc ids and where information has not been captured we redefined
    #       the patterns. i.e appended
        
    tags = df_bd.type_tag.values
    per_vals = df_bd.per_value.values
    words=df_bd.word.values
    
    # Searching for key patterns...
    found_pattern = False
    found_key_patterns = dict.fromkeys(key_patterns.keys()) # to be appended to with our found items
    for key in key_patterns.keys(): # iterates through keys
        found_key_patterns[key] = [] # sets our found dict to contain lists so we can append

    for key in key_patterns.keys():
        for marker in key_patterns[key]: # iterates through strings to match in each key
            tags_joined = ' '.join(tags)

            while marker in tags_joined:  
                if key in ['VR','NOC']:
                    found_pattern = True
                marker_arr = np.array(marker.split(' ')) # turns to array for indexing 
                inds_of_marker = search_sequence_numpy(tags,marker_arr) # searches for indices of sequence in original

                # Checking single sequence... and reducing if so
                for i in range(1,len(inds_of_marker)):
                    if inds_of_marker[i]-1 != inds_of_marker[i-1]:
                        inds_of_marker = inds_of_marker[:i]
                        break
                
                
                if key == 'SIC': # if SIC we dont search for digits
                    if 'NEGATIVE' in marker:
                        found_key_patterns[key] = [False] if marker.index('NEGATIVE') < marker.index('SIC') else [True]
                    else:
                        found_key_patterns[key] = [True]
    
                    tags = np.delete(tags,inds_of_marker)
                    words = np.delete(words,inds_of_marker)
                    tags_joined = ' '.join(tags)
                    continue
    
                digit_locs = np.where(marker_arr == 'DIGIT')
                ind_digit = [inds_of_marker[k] for k in digit_locs] # then gets indices of digit markers
                digits = per_vals[ind_digit] # returns the percentages that are being spoke about
                
                if 'LESSER' in marker:
                    if marker.index('LESSER') < marker.index('GREATER'):
                        digits = [digits[1],digits[0]]
                
                noc_and_vr = False
                if key == 'NOC':
                    if 'VOTING' in marker.split(' '):
                        noc_and_vr = True
                
                if noc_and_vr == True:
                    found_key_patterns[key].append([marker,list(digits)]) 
                    found_key_patterns['VR'].append([marker,list(digits)]) 
                else:
                    found_key_patterns[key].append([marker,list(digits)]) # appends found items to dict
                            

                # reducing dataset so multiple matches aren't confused
                tags = np.delete(tags,inds_of_marker)
                words = np.delete(words,inds_of_marker)
                per_vals = np.delete(per_vals,inds_of_marker)
                
                # for checking again...
                tags_joined = ' '.join(tags)
                

    #print(found_key_patterns)
    if found_pattern == True:
        # Determining which categories have been found...
        found_tags = []
        for key in found_key_patterns.keys():
            if len(found_key_patterns[key]) > 0:
                found_tags.append(key)
    else:
        # checking if voting is mentioned, if not then we assume they are mentioning share ownership
        if ('VOTING' not in tags):
            if ('DIGIT' in tags) or ('PERCENT' in tags):
                nums_extract = df_bd[~df_bd.per_value.isna()].per_value.values
                found_pattern = True if len(nums_extract) > 0 else found_pattern
                # grouping those that should go together...
                ordering = [25,50,75]
                i = 0
                while i < len(nums_extract):
                    current_num = nums_extract[i]
                    if current_num in ordering:
                        num_ind = ordering.index(current_num)
                        
                        if i < len(nums_extract) - 1:
                            if nums_extract[i+1] in ordering: # i.e
                                num_2_ind = ordering.index(nums_extract[i+1])
                            else:
                                found_key_patterns['NOC'].append([None,[current_num]]) # appends in correct format
                                i += 1
                                continue
                        
                            if num_2_ind == num_ind + 1:
                                found_key_patterns['NOC'].append([None,[current_num,nums_extract[i+1]]]) # the case where a range has been found
                                i += 2
                                continue
                            else:
                                found_key_patterns['NOC'].append([None,[current_num]])
                                i += 1
                                continue
                    
                    found_key_patterns['NOC'].append([None,[current_num]])
                    i += 1
                    continue
                    
        found_tags = []
        for key in found_key_patterns.keys():
            if len(found_key_patterns[key]) > 0:
                found_tags.append(key)
    
    # checking if all items have been caught...
    # we only want to do this when single type is mentioned as multi types should capture but also cause confusion
    
    if len(found_tags) <= 1:
        per_vals = [per for per in per_vals if (per>10 and per <=100)]
        tag_to_use = found_tags[0] if len(found_tags) == 1 else 'NOC'
        
        for per in per_vals:
            found_key_patterns[tag_to_use].append([None,[per]])
                        
    to_return_dict = dict.fromkeys(found_key_patterns.keys())
    for key in to_return_dict:
        to_return_dict[key] =[]
    print(found_key_patterns)
    # Now we have our found tags we must check these controls are on the register for the individual,
    # Note we first check we have a singular person being discussed
    df_noc_red.merged_name = df_noc_red.merged_name.apply(lambda x: is_psc(x,sh_in_txt[0]))

    if count_found == 1:
        df_psc = df_noc_red[df_noc_red.merged_name == sh_in_txt[0].lower()] if len(df_noc_red.merged_name.unique()) > 1 else df_noc_red
        
        # exit here is due to more than 3 controls listed which is not allowed
        if df_psc[df_psc.merged_name == sh_in_txt[0].lower()].shape[0] > 3:
            chips_controls = get_controls(df_psc,multi = True)
            
            for key in ['NOC','VR']:
                if len(found_key_patterns[key]) == 0:
                    found_key_patterns[key] = None
                else:
                    new = []
                    for i in found_key_patterns[key]:
                        new.append(i[1])
                    found_key_patterns[key] = new
                            
            return ['TOO MANY CONTROLS - HIGH',format_chips_op(found_key_patterns), format_chips_op(chips_controls)],df_noc_red
        else:
            # here we want to state the shareholders controls
            chips_controls = get_controls(df_psc)
            
        
        cats = df_psc.TYPE.unique()
        
        difference_found = False
        difference_found_in = [] # for approapriate priority in output
        # now we go through found tags
        for tag in found_tags:
            if tag not in cats:
                items = found_key_patterns[tag]
                to_add = [item[1] for item in items] if tag != 'SIC' else items[0]
                to_return_dict[tag].append(to_add)
                difference_found = True
                difference_found_in.append(tag)
                
            elif tag != 'SIC':
                # getting lower and upper bounds...
                bounds = df_psc[df_psc.TYPE == tag][['LB','UB']].values[0]
                
                if bounds[1] is np.nan: # if we have top cat i.e 75% or more
                    bounds[1] = 100
                
                to_check = found_key_patterns[tag]
                
                for items in to_check: # checks through all found patterns
                    item = items[1]
                    if len(item) == 1:
                        if item[0] >= 75:
                            if bounds[0] != 75:
                                to_return_dict[tag].append(item)
                                difference_found = True
                                difference_found_in.append(tag)
                            else:
                                continue
                        elif item[0] == 25:
                            continue
                        else:
                            if (item[0] > bounds[0]) and (item[0] < bounds[1]):
                                continue
                            else:
                                to_return_dict[tag].append(item)
                                difference_found = True
                                difference_found_in.append(tag)
                    else:
                        if (item[0] == bounds[0]) and (item[1] == bounds[1]):
                            continue
                        elif (item[0] == bounds[0]) and (item[1] == 100):
                            continue
                        else:
                            to_return_dict[tag].append(item)
                            difference_found = True
                            difference_found_in.append(tag)
            elif tag == 'SIC':
                to_return_dict[tag] = found_key_patterns[tag]
    

    # checking if difference in sic
    if (found_key_patterns['SIC'] != []) and (chips_controls['SIC'] != found_key_patterns['SIC'][0]):
        difference_found = True
        
    if (limited_tag == True) and (chips_controls['NOC'] != None):
        return ['LIMITED BY GUARENTEE - HIGH',None,format_chips_op(chips_controls)],df_noc_red
    
                
    if difference_found == True:
        for key in ['NOC','VR']:
                if len(to_return_dict[key]) == 0:
                    to_return_dict[key] = None
                else:
                    #checking successive list
                    if len(to_return_dict[key]) == 1:
                        if type(to_return_dict[key][0]) == list:
                            to_return_dict[key] = to_return_dict[key][0][0] if type(to_return_dict[key][0][0]) == list else to_return_dict[key][0]

        # checking where differences are...
        if 'NOC' in difference_found_in:
            tag_to_give = ' SHARES - HIGH'
        else:
            tag_to_give = ' VOTING OR RTA - LOW' if limited_tag != True else ' VOTING OR RTA - HIGH' 
        
        return ['DIFFERENCE FOUND' + tag_to_give,format_chips_op(to_return_dict),format_chips_op(chips_controls)],df_noc_red
    else:
        # checking patterns have been found...
        if found_pattern == True:
            # preping the return dict
            for key in ['NOC','VR']:
                if len(to_return_dict[key]) == 0:
                    to_return_dict[key] = None
                else:
                    #checking successive list
                    if len(to_return_dict[key]) == 1:
                        if type(to_return_dict[key][0]) == list:
                            to_return_dict[key] = to_return_dict[key][0][0] if type(to_return_dict[key][0][0]) == list else to_return_dict[key][0]
            
            # checking trustee, firm, person
            # first get the type for NOC and voting
            df_psc = df_noc_red[df_noc_red.merged_name == sh_in_txt[0].lower()]
            noc_type = list(set([item for item in ['trustee','person','firm'] if item in ' '.join(df_psc.DESCRIPTION.values).lower()]))
            noc_type_in_txt = list(set([item for item in ['trustee','person','firm'] if item in df[df.PSC_DISCREPANCY_ID == psc_id].discrepancy_detail.iloc[0].lower() ]))
            
            trigger_noc = False
            for item in noc_type_in_txt:
                if item not in noc_type:
                    trigger_noc = True
                    
            if (len(noc_type) > 1) or (trigger_noc == True):
                return ['TRUSTEE, PERSON, OR FIRM DIFFERENCE - HIGH',format_chips_op(to_return_dict),format_chips_op(chips_controls)],df_noc_red
            
            return ['MATCHED - NONE',format_chips_op(to_return_dict),format_chips_op(chips_controls)],df_noc_red
        else:
            # Checking for removed tags
            txt = df[df.PSC_DISCREPANCY_ID == psc_id].discrepancy_detail.iloc[0].lower()
            rem_tags = ['since been removed','please remove','no longer psc','no longer a psc','remove psc','removal of psc','psc removal','no longer psc','no longer a shareholder','remove this person']
            for item in rem_tags:
                if (item in txt) or (rem_tag == True):
                    return ['REMOVAL - HIGH',psc_name,psc_name],df_noc_red
            
            
            # checking controls and stated controls where no patterns found and no % values extracted
            count_of_tags = df_bd.type_tag.value_counts()
            NOC = count_of_tags['OWNERSHIP'] if 'OWNERSHIP' in count_of_tags.index else 0
            VR = count_of_tags['VOTING'] if 'VOTING' in count_of_tags.index else 0
            SIC = count_of_tags['SIC'] if 'SIC' in count_of_tags.index else 0
            
            NOC = True if NOC > 0 else False
            VR = True if VR > 0 else False
            SIC = True if SIC > 1 else False # this is 1 as SIC always mentioned twice for SIC cases but can be once for non SIC
            
            chips_NOC = True if chips_controls['NOC'] != None else False
            chips_VR = True if chips_controls['VR'] != None else False
            chips_SIC = True if chips_controls['SIC'] == True else False
            
            chips = [chips_NOC,chips_VR,chips_SIC]
            text_ex = [NOC,VR,SIC]
            
            if (chips == [True,False,True] ) and (VR == True) : # reported VR missing, holds NOC and SIC in CHIPS
                return ['WITHOUT VOTING RIGHTS - LOW',None,format_chips_op(chips_controls)],df_noc_red
            
            elif (chips == [False,False,True]) and (VR == True) and (NOC == False): # reported VR missing, holds SIC only
                return ['WITHOUT VOTING RIGHTS (SIC IN CHIPS) - HIGH',None,format_chips_op(chips_controls)],df_noc_red
            
            elif (NOC == True) and (chips[0] == False): # NOC present but not in CHIPS
                return ['WITHOUT SH - HIGH',None,format_chips_op(chips_controls)],df_noc_red
            
            elif (VR == True) and (sum(chips) == 0):
                return ['WITHOUT VOTING RIGHTS - LOW',None,format_chips_op(chips_controls)],df_noc_red
            
            elif ('remove' in df_bd.word.values) or ('removed' in df_bd.word.values):
                return ['REMOVAL - HIGH',None,format_chips_op(chips_controls)],df_noc_red
            
            elif ((NOC == True) and (chips[0] == True)) or ((VR == True) and (chips[1] == True)): # have we found missing logic
                return ['MISSING CONTROLS FOUND - NONE',None,format_chips_op(chips_controls)],df_noc_red
            
            return ['INSUFFICIENT INFO - LOW',None,format_chips_op(chips_controls)],df_noc_red # if none of the above...
                
    return ['tag',found_key_patterns], df_noc_red
    

In [ ]:
def noc_prio(psc_id,df):
    print("CURRENT ID: ",psc_id)
    df_bd,psc_name,rem_tag,lim_tag = text_breakdown(df,psc_id)
    op,df_noc_red = breakdown_to_op(psc_id,df_bd,df,psc_name,rem_tag,lim_tag)
    
    index = df[df.PSC_DISCREPANCY_ID == psc_id].index[0]
    
    
    df.loc[index,'priority_explained'] = op[0].split(' - ')[0]
    df.loc[index,'priority'] = op[0].split(' - ')[1]
    df.loc[index,'material_nonmaterial'] = 'MATERIAL' if 'NONE' not in op[0].split(' - ')[1] else 'NON-MATERIAL'

    if 'REMOVAL' in op[0]:
        df.loc[index,'CHIPS_value'] = op[1]
        return op[2]
    
    else:
        df.loc[index,'CHIPS_value'] = str(op[2]).replace('[]','None').replace('{','').replace('}','').replace(']','%').replace('[','').replace(',','% -').replace('None%','None').replace('nan%','100%').replace('%%','%').replace("'%","'").replace('%%','%').replace('True%','True').replace('False%','False').replace('%%','%')
        return str(op[1]).replace('[]','None').replace('{','').replace('}','').replace(']','%').replace('[','').replace(',','% -').replace('None%','None').replace('nan%','100%').replace('%%','%').replace("'%","'").replace('%%','%').replace('True%','True').replace('False%','False').replace('%%','%')

# NATIONALITY

In [ ]:
def nationality_compare(psc_id,df, trace = False):
    #print("\n=======================================================================================================================")
    # fetching discrepancy info
    txt = df[df.PSC_DISCREPANCY_ID == psc_id].discrepancy_detail.iloc[0]#.lower()
    psc_name = df[df.PSC_DISCREPANCY_ID == psc_id].extract_psc_name.iloc[0].lower()
    cont_id = df[df.PSC_DISCREPANCY_ID == psc_id].CONTACT_ID.iloc[0]
    comp_num = df[df.PSC_DISCREPANCY_ID == psc_id].COMPANY_NUMBER.iloc[0]
    corp_id = df[df.PSC_DISCREPANCY_ID == psc_id].CORPORATE_BODY_ID.iloc[0]
    
    if trace == True:
        print("TEXT : ",txt)
        print("NAME : ",psc_name)
        print("CONTACT : ",cont_id)
    
    
    #checking for NOC mentions
    noc_buzzwords = ['nature of control','ownership','shares','voting', '%']           
    noc_found = any([True if x in txt.lower() else False for x in noc_buzzwords]) == True
    
    # fetching SH info
    df_cont = df_off[df_off.CONTACT_ID == cont_id]
    
    # preprocessing text
    remove_punc = ['\n','\r','!','.',',','"',"'",'(',')',';',':','-','&']
    for punc in remove_punc:
        txt = txt.replace(punc,' ')
        
    txt_split = txt.split(' ')
    while '' in txt_split:
        txt_split.remove('')
    
    
    # get pos tagging
    pos_tags = get_pos_tags(txt,nlp,nationalities = nationalities,reduce_ents = False, nat = True)
    
    norp_items = list(set(pos_tags['NORP']))
    
    for nat in nationalities:
        nat = nat.lower()
        while nat[0] == ' ':
            nat = nat[1:]
        while nat[-1] == ' ':
            nat = nat[:-1]
            
        if (nat in txt.lower()) and (nat not in norp_items):
            norp_items.append(nat)
    
    # sometimes country captured as ORG, adding these to selection
    org_items = list(set(pos_tags['ORG']))

    for item in org_items:
        item = item.lower()
        if (item in nationalities) and (item not in norp_items):
            norp_items.append(item)
                
    # the sri lanka check as spacy doesnt like picking this up
    if ('sri lanka' in txt.lower()) or ('sri lankan' in txt.lower()):
        if ('sri lanka' not in norp_items) or ('sri lankan' not in norp_items):
            norp_items.append('sri lanka')
            
    # changing all to most common adjectival for country
    reduced_items = []
    for norp in norp_items:
        norp = norp.lower()
        if norp in df_nat_and_adj.country.values:
            new_name = df_nat_and_adj[df_nat_and_adj.country == norp].adjectivals.values[0]
            reduced_items.append(new_name)
            
        elif norp in df_nat_and_adj.adjectivals.values:
            our_country = df_nat_and_adj[df_nat_and_adj.adjectivals == norp].country.values[0]
            new_name = df_nat_and_adj[df_nat_and_adj.country == our_country].adjectivals.values[0]
            reduced_items.append(new_name)
            
        else:
            reduced_items.append(norp)
            
            
    reduced_items = list(set(reduced_items))
    norp_orig = reduced_items.copy()
    
    #print("\nEXTRACTED NORPS: ",reduced_items)
    
    # get sh
    our_sh = get_sh_from_shs(psc_name,df_cont[~df_cont.merged_name.isna()].merged_name.unique())
    
    if our_sh == None: # if no person found
        return ["PERSON NOT FOUND - HIGH",['None'],reduced_items]
    
    our_sh_id = df_cont[df_cont.merged_name == our_sh].OFFICER_ID.values[0]
    
    # removing nationalities stored from nationalities extracted
    our_sh_nat = df_cont[df_cont.OFFICER_ID == our_sh_id].OFFICER_NATIONALITY.apply(lambda x: x.lower() if type(x) == str else '').unique()
    our_sh_nat = list(our_sh_nat)
    while '' in our_sh_nat:
        our_sh_nat.remove('')
        
    new_nat_sh = []

    # for when dual nationality is stored as e.g. "british,nigerian"
    for nat in our_sh_nat:
        if ',' in nat:
            nat_split = nat.split(',')
            for nat in nat_split:
                new_nat_sh.append(nat)
        else:
            new_nat_sh.append(nat)
            
    our_sh_nat = list(set(new_nat_sh))

    #print("SH NAT: ",our_sh_nat)
    for nat in our_sh_nat: 
        if nat in reduced_items:
            reduced_items.remove(nat)

    # checking dual nationality
    if 'dual' in txt.lower():
        if (len(our_sh_nat) == 0) or (len(reduced_items) > 0): # if no nationalities listed but dual is mentioned
            return ['NOT MATCHED - LOW' if noc_found == False else 'NOC FOUND - HIGH',our_sh_nat,reduced_items]
        else: # if mentioned but matched
            return ['DUAL NATIONAL MATCHED - NONE' if noc_found == False else 'NOC FOUND - HIGH',our_sh_nat,reduced_items]
    
    #print("REMAINING NATS: ",reduced_items)
    
    if len(reduced_items) > 0: # remaining items that have not been matched
        return ['NOT MATCHED - LOW' if noc_found == False else 'NOC FOUND - HIGH',our_sh_nat,reduced_items]
    
    elif len(norp_orig) == 0: # if none of the above and none extract then insufficient info to procede
        
        return ['INSUFFICIENT INFO - LOW' if noc_found == False else 'NOC FOUND - HIGH',our_sh_nat,reduced_items]
    
    else: # if none of the above then we know
        return ['MATCHED - NONE' if noc_found == False else 'NOC FOUND - HIGH',our_sh_nat,reduced_items]
    
    
    return ['ERROR',None,None]



In [ ]:
def nationality_prio(psc_id,df):
    print(psc_id)
    index = df[df.PSC_DISCREPANCY_ID == psc_id].index
    op = nationality_compare(psc_id,df)
    
    df.loc[index,'priority_explained']  = op[0].split(' - ')[0]
    df.loc[index,'priority'] = op[0].split(' - ')[1]
    df.loc[index,'CHIPS_value'] = ', '.join(op[1])
    df.loc[index,'material_nonmaterial'] = 'NON-MATERIAL' if 'NONE' in op[0] else 'MATERIAL'
    return ', '.join(op[2]) # returns text_difference

# DOB

In [ ]:
def get_dobs_from_text(psc_id,df_rel,trace = False):
    # first check action code...
    cont_id = df_rel[df_rel.PSC_DISCREPANCY_ID == psc_id].CONTACT_ID.values[0]
    
    # Fetching Data
    txt_orig = df_rel[df_rel.PSC_DISCREPANCY_ID == psc_id].discrepancy_detail.values[0]
    txt = txt_orig.lower()
    
    comp_num = df_rel[df_rel.PSC_DISCREPANCY_ID == psc_id].COMPANY_NUMBER.values[0]
    psc_name = df_rel[df_rel.PSC_DISCREPANCY_ID == psc_id].PSC_NAME.values[0]
    corp_id = df_rel[df_rel.PSC_DISCREPANCY_ID == psc_id].CORPORATE_BODY_ID.values[0]
    
    if type(psc_name) != str:
        return [None,None,'NO PSC NAME - NONE']
    else:
        psc_name = psc_name.lower()
    
    if trace == True:
        print("\n========================================================================================================================================================")
        print("PSC ID : ",psc_id)
        print("PSC Name : ",psc_name)
        print("COMPANY NUMBER : ",comp_num)
        print("CORPORATE ID : ",corp_id)
        print("CONTACT ID : ",cont_id)
        print("\nThe text : ")
        print(txt_orig)

    # minor preprocessing
    remove_punc = ['\n','\r','!','.',',','year','"',"'",'(',')','dob','ch','month',';',':','-']
    for punc in remove_punc:
        txt = txt.replace(punc,' ')
    
    txt_split = txt.split(' ')
    while '' in txt_split:
        txt_split.remove('')
    
    # spacy date extraction
    pos_tags = get_pos_tags(txt,nlp,reduce_ents = True)
    pos_in_txt = [assign_pos(pos_tags,x) for x in txt_split]
    
    # Setting up date extraction...
    month_names = {'january':'01','february':'02','march':'03','april':'04','may':'05','june':'06','july':'07','august':'08','september':'09','october':'10',
                  'november':'11','december':'12'}
    short_months = {month[:3]:v for month,v in month_names.items() }

    end_of_nums = ['st ','nd ','rd ','th '] # for 1st 2nd 3rd 4th replacement to 1,2,3,4
    
    
    # turning txt into df
    df_txt = pd.DataFrame({'words':txt_split,'pos':pos_in_txt})
    df_txt['tags'] = ['DIGIT' if x.isdigit() else 'MONTH' if x in month_names.keys() else 'MONTH SHORT' if x in short_months.keys() else np.nan for x in txt_split  ]
    df_txt.loc[df_txt.tags == 'MONTH','words'] = [month_names[x] for x in df_txt[df_txt.tags == 'MONTH'].words.values]
    df_txt.loc[df_txt.tags == 'MONTH SHORT','words'] = [short_months[x] for x in df_txt[df_txt.tags == 'MONTH SHORT'].words.values]
    df_txt.loc[df_txt.tags == 'MONTH SHORT','tags'] = 'MONTH'
    df_txt.loc[df_txt.tags == 'DIGIT','tags'] = df_txt.loc[df_txt.tags == 'DIGIT','words'].apply(lambda x: 'YEAR' if len(x) == 4 else 'DIGIT')
    df_txt.loc[df_txt.tags == 'YEAR','words'] = df_txt.loc[df_txt.tags == 'YEAR','words'].apply(lambda x: int('19'+str(x)) if len(x) == 2 else x)
    
    # checking surrounding indices if YEAR - MONTH - DAY formatting
    year_inds = df_txt[df_txt.tags == 'YEAR'].index
    for ind in year_inds:
        if ind+1 < df_txt.shape[0]:
            tag = df_txt.loc[ind+1,'tags'] 
            if tag == 'DIGIT':
                val = int(df_txt.loc[ind+1,'words'])
                if (val <= 12) and (val > 0):
                    df_txt.loc[ind+1,'tags'] = 'MONTH'
                    
    # final sweep for missed dates...
    for i in range(df_txt.shape[0]):
        word = df_txt.loc[i,'words']
        
        if '/' in word:
            word_split = word.split('/')
        elif '-' in word:
            word_split = word.split('-')
        else:
            continue
        
        if all([part.isdigit() for part in word_split]) == True:
            df_txt.loc[i,'pos'] = 'DATE'
    
    df_txt['to_keep'] = [False if (type(x) != str) and (type(y) != str) else True for x,y in df_txt[['pos','tags']].values]
    df_txt = df_txt[df_txt.to_keep == True][['words','pos','tags']]
    df_txt =df_txt.reset_index(drop=True)
    
    whole_dates = df_txt[(df_txt.tags.isna()) & (df_txt.pos == 'DATE')].words.values
    months = list(df_txt[df_txt.tags == 'MONTH'].words.values)
    years = list(df_txt[df_txt.tags == 'YEAR'].words.values)
    
    #print(whole_dates)
    for date in whole_dates:
        date = date.replace('-',' ').replace('/',' ')
        date_split = date.split(' ')
        
        if len(date_split) == 1:
            continue
        
        year_last = True
        

        if (len(date_split[0]) == 4):
            years.append(date_split[0])
            year_last = False
        elif date_split[0].isdigit():
            if  (int(date_split[0]) > 31):
                years.append(date_split[0])
                year_last = False
            else:
                years.append(date_split[-1])
            
        if year_last == True:
            months.append(date_split[-2])
        else:
            months.append(date_split[1])
        
    # check if month only or year only or both
    type_of_rep = 'Both' if (len(months)*len(years) > 0) else 'Year' if len(years) > 0 else 'Month'
    
    # fetch sh information
    for title in ['dr','mr','mrs','mx','ms']:
        psc_name = psc_name.replace(title,'')
    
    sh_info = df_off[df_off.CONTACT_ID == cont_id].reset_index(drop=True)
    if sh_info.shape[0] == 0:
        return [None, None, 'NO CHIPS VALUE FOUND - HIGH']
    
    sh_info['OFFICER_DATE_OF_BIRTH'] = pd.to_datetime(sh_info['OFFICER_DATE_OF_BIRTH'],errors = 'coerce')
    sh_info = sh_info.fillna('')
    sh_info['merged_name'] = [(sh_info.loc[i,'OFFICER_FORENAME_1'] +' '+ sh_info.loc[i,'OFFICER_FORENAME_2'] +' '+ sh_info.loc[i,'OFFICER_SURNAME']).lower() for i in range(0,sh_info.shape[0])]
    
    # selecting relevant sh
    shs = sh_info.merged_name.unique()
    sh_scores = np.zeros_like(shs)
    psc_split = psc_name.split(' ')
    for i in range(0,len(shs)) :
        sh = shs[i]
        sh_split = sh.split(' ')
        for part in psc_split:
            if part in sh_split:
                sh_scores[i] +=1
    
    our_sh = get_sh_from_shs(psc_name,sh_info[~sh_info.merged_name.isna()].merged_name.unique())
    our_sh_id = sh_info[sh_info.merged_name == our_sh].OFFICER_ID.values[0]
    
    our_dob = sh_info[sh_info.OFFICER_ID == our_sh_id].OFFICER_DATE_OF_BIRTH.unique()
    if len(our_dob) > 1:
        to_return_dob = []
        for dob in our_dob:
            to_return_dob.append(str(dob)[:10])
        
        the_issue = {'month':months ,'year':years }
        return [to_return_dob,the_issue,'MANY DOB SINGLE OFFICER - HIGH']
    
    our_dob = our_dob[0] if len(our_dob) == 1 else our_dob
    
    our_mon = our_dob.astype('datetime64[M]').astype(int) %12+1
    our_year = our_dob.astype('datetime64[Y]').astype(int) +1970

    if int(str(our_year)[-2:]) > 22:
        our_year = int('19'+str(our_year)[-2:])
    our_dob = str(our_year)+'-'+str(our_mon) if len(str(our_mon)) == 2 else str(our_year)+'-0'+str(our_mon)
    
    # checking differences
    months_issue = False
    years_issue = False
    issue_mon = []
    issue_yr = []
    for mon in months:
        if mon.isdigit() == False:
            continue
        if (int(mon) != our_mon) and (type(mon) != None):
            months_issue = True
            issue_mon.append(mon)
    
    for year in years:
        if year.isdigit() == False:
            continue
        if (int(year) != our_year) and (type(year) != None):
            years_issue = True
            issue_yr.append(year)
 
    
    the_issue = {'month':issue_mon ,'year':issue_yr }
    issue = 'DIFFERENCE FOUND - HIGH' if (months_issue == True) or (years_issue == True) else 'MATCHED - NONE'

    return [our_dob,the_issue,issue] # op of order : op[0] = CHIPS_value, op[1] = text_difference, op[2] = tells us if there is an issue or not


In [ ]:
def dob_prio(psc_id,df):
    print("CURRENT ID: ",psc_id)

    index = df[df.PSC_DISCREPANCY_ID == psc_id].index[0]
    
    op = get_dobs_from_text(psc_id,df)

    # Formatting output
    chips = op[0]
    diff = op[1]
    tag = op[2]
    
    tag_explained = tag.split(' - ')[0]
    tag = tag.split(' - ')[1]
    
    if type(chips) == list:
        chips = ', '.join(chips)
        
    if type(diff) == dict:
        op_diff = ''
        for key in diff.keys():
            op_diff = op_diff + key + ': ' 
            op_diff = op_diff + ', '.join(diff[key]) + ' ' if len(diff[key]) > 0 else op_diff + 'None '
        diff = op_diff
    
    df.loc[index,'CHIPS_value'] = chips
    df.loc[index,'priority'] = tag
    df.loc[index,'priority_explained'] = tag_explained
    df.loc[index,'material_nonmaterial'] = 'MATERIAL' if 'NONE' not in op[2] else 'NON-MATERIAL'
    
    return diff 

# OTHERS & PSC MISSING

In [ ]:
# extract the listed Discrepancies from the section
def Discrepancy(x):
    txt = re.search('Discrepancy Options:(\s+)(.)*', x)
    myText = txt.group(0).replace('Discrepancy Options:\n ', '')
    ls = myText.split(',')
    return ls

# find the material types of discrepancy within List of Discrepancies and assign Multi, Multi_NoNoc & Single
def find_noc(x,optionList):
    if len(x) > 1:
        matching = [s for s in x if any(w in s for w in optionList)]
        if matching:
            return 'Multi'  # When material types of Discrepancies listed within Options
        else:
            return 'Multi-Low'  # Multiple discrepancy listed but only non-material types of discrepancy
    else:
        return 'Single'  # Only one types of discrepancy listed (material/non material)
 

    
# function to assign Material and Non-Material to discrepancies
def material_nonmaterial(x):
    # material types deiscrepancy
    if x == 'Governing law' or x == 'Incorporation law' or x == 'Legal form' or x == 'Place of registration' or x == 'Place of residence' or x == 'Correspondence address' or x == 'Other reason' or x == 'Company number':
        return 'MATERIAL'
    # non-material types
    elif x == 'Notified date' or x == 'Country of residence' :
        return 'NON-MATERIAL'
    else:
        return None
    
#  function to assign High/Low priority
def priority(x):
    # print(x)
    # when option list contains any of high priority discrepancy
    if x == 'Multi':
        return 'HIGH'
    # otherwise for the rest of discrepancy it will be low
    else:
        return 'LOW'
    
# function to find company number
def compnum(x):
    x = x.replace('\n', ' ')
    txt = re.search('Discrepancy Details:(.)*Discrepancy Options:', x)
    cnum = re.search(r'\d{5,12}', txt.group(0))
    return cnum.group(0)

# function to assign priority : compnay number df
def compnum_priority(x):
    if x['DISCREPANCY_COMPANY_NUMBER'] == x['text_difference']:
        return 'Low'
    else:
        return 'High'
    
# find the different  types of discrepancy from the Other discrepancy while discrepancies are not listed clearly
def othercontent(x):
    x = x.lower().replace('\n', ' ')
    txt = re.search('discrepancy details:(.)*discrepancy options:', x)
    mytext = txt.group(0)
    # when person cesed or died
    if 'ceased' in mytext or 'died' in mytext:
        return 'Cease'
    elif 'not a rle' in mytext or 'shouldn' in mytext or 'should not' in mytext or 'duplicat' in mytext or 'twice' in mytext or 'removed' in mytext or 'remove' in mytext or 'terminate' in mytext or 'not be listed' in mytext or 'should not be considered' in mytext or 'no ownership' in mytext or 'no longer' in mytext or 'not a psc' in mytext:
        # print(mytext)
        return 'PSC Remove'
    # when PSC are still active/asked for update/to add the PSC
    elif 'is still the active' in mytext or 'in addition' in mytext or 'still active' in mytext or 'added' in mytext or 'not updated' in mytext:
        return 'PSC Add'
    # when PSC complained about the missing VR
    elif ('missing' in mytext or 'without' in mytext)  and 'voting rights' in mytext:
        return 'Missing VR'
    # when PSC asked for Share and Voting rights
    elif '%' in mytext or 'share' in mytext or 'voting' in mytext or 'influence' in mytext or 'control' in mytext:
        return 'Share & VR'
    # when PSC asked for both addition and removal of PSC from the register
    elif (
             'not a rle' in mytext or 'shouldn' in mytext or 'should not' in mytext or 'duplicat' in mytext or 'twice' in mytext or 'removed' in mytext or 'remove' in mytext or 'terminate' in mytext or 'not be listed' in mytext or 'should not be considered' in mytext or 'no ownership' in mytext or 'no longer' in mytext or 'not a psc' in mytext)\
            and \
            ('is still the active' in mytext or 'added' in mytext or 'in addition' in mytext or 'still active' in mytext or 'not updated' in mytext):

        return 'Both Addition and Removal'
    # when PSC asked for both addition and removal of PSC from the register
    elif 'is the psc and not' in mytext:
        return 'Both Addition and Removal'
    # when PSC are inactive/duplicate entry/ask for removal/termination of PSC
    elif 'trust' in mytext: # about the trust
        return 'Trust'
    elif 'dob' in mytext: # when user talk about the DOB
        return 'DOB'
    elif 'incorrect' in mytext: # section describe incorrect details
        return 'Incorrect details'
    else:
        return 'Miscellaneous'

# mark in the Other reason discrepancy 'cease' as Non-Material and rest as Material
def material(x):
    # print(x)
    if x == 'Cease':
        return 'NON-MATERIAL'
    else:
        return 'MATERIAL'
    
def queother(x):
    if (x['priority_explained'] == 'Missing VR'):
        return 'LOW'
    else:
        return 'HIGH'
    
# assign the discrepancy to Other section based on text extracted from the above function
def detailcontent(x):
    x = x.replace('\n', ' ')
    txt = re.search('Discrepancy Details:(.)*Discrepancy Options:', x)
    mytext = txt.group(0)
    return mytext

In [ ]:
def other_cat_prio(df):

    # Sub data frames :
    dfglaws = df[df['PSC_DISCREPANCY_OPTION_DESC'] == 'Governing law'] # Governing Law dataframe
    dfinlaw = df[df['PSC_DISCREPANCY_OPTION_DESC'] == 'Incorporation law'] # ncorporation law dataframe
    dflegal = df[df['PSC_DISCREPANCY_OPTION_DESC'] == 'Legal form'] # Legal form dataframe
    dfndate = df[df['PSC_DISCREPANCY_OPTION_DESC'] == 'Notified date'] # Notified date dataframe
    dfcr = df[df['PSC_DISCREPANCY_OPTION_DESC'] == 'Country of residence'] # Country of residence dataframe
    dfpr = df[df['PSC_DISCREPANCY_OPTION_DESC'] == 'Place of residence'] # lace of residence dataframe
    dfpres = df[df['PSC_DISCREPANCY_OPTION_DESC'] == 'Place of registration'] # Place of registration dataframe
    dfaddr = df[df['PSC_DISCREPANCY_OPTION_DESC'] == 'Correspondence address'] # Correspondence address dataframe
    dfcomp = df[df['PSC_DISCREPANCY_OPTION_DESC'] == 'Company number'] # Company number dataframe
    
    df_frames = [dfglaws, dfinlaw, dflegal, dfndate, dfpr, dfpres,dfaddr,dfcomp] # for merging of the data frames
    
    df_result = pd.concat(df_frames) # concatenating dataframes
    
    #  Tag the whole data with Material/Non Material
    df_result['material_nonmaterial'] = df_result['PSC_DISCREPANCY_OPTION_DESC'].apply(material_nonmaterial)
    
    #  assign priorities to whole df becoz these discrepancies would be either High: Multi or Low
    df_result['priority'] = df_result['Discrepancy'].apply(priority)
    
    #  data frame for the 'Other reason'  discrepancy
    dfother = df[df.PSC_DISCREPANCY_OPTION_DESC == 'Other reason']

    # extract text from the other reason data frame
    dfother['priority_explained'] = dfother['CONTACT_DETAIL_CLOB'].apply(othercontent)
    dfother['material_nonmaterial'] = 'MATERIAL'
    
    dfother['priority'] = dfother.apply(queother,axis=1)
    
    # country of residence all to NONE
    dfcr['materia_nonmaterial'] = 'NON-MATERIAL'
    dfcr['priority'] = 'NONE'
    dfcr['priority_explained'] = 'NON-MATERIAL PSC TYPE'
    
    frames = [df_result,dfother,dfcr]
    
    result = pd.concat(frames)
    
    #  Discrepancy detail section
    result['discrepancy_detail'] = result['CONTACT_DETAIL_CLOB'].apply(detailcontent)
    result['priority_explained'][result.priority_explained.isna()] = result[result.priority_explained.isna()].material_nonmaterial.apply(lambda x: 'NON-MATERIAL PSC TYPE' if x == 'NON-MATERIAL' else 'MATERIAL PSC TYPE')
    result[df['PSC_DISCREPANCY_OPTION_DESC'] == 'Company number']['priority'] = 'HIGH'
    
    return result

# PSC Missing Logic

In [ ]:
def check_missing(psc_id,df):
    #if 'should be listed' or 'should be a psc' or 'no one listed' or 'should be on psc' or 'they should disclose' or 'is the psc' or 'the psc is' or 'the psc as' or 'psc register is empty' or '' 
    index = df[df.PSC_DISCREPANCY_ID == psc_id].index[0]
    txt = df.loc[index,'discrepancy_detail']
    cont_id = df.loc[index,'CONTACT_ID']
    corp_id = df.loc[index,'CORPORATE_BODY_ID']
    comp_name = df.loc[index,'SURNAME']
    print(comp_name)
    
    # getting shareholder names
    df_cont = df_off[df_off.CONTACT_ID == cont_id]
    
    if df_cont.shape[0] == 0:
        return 'PSC Missing'
    
    sh_names = df_cont.merged_name.unique()
    
    sh_names = [name.lower() for name in sh_names]
    sh_fores = [name.split(' ')[:-1] for name in sh_names]
    
    print("SH NAMES : ",sh_names)
    sh_surs = [name.split(' ')[-1] for name in sh_names]
    
    pos_tags = get_pos_tags(txt,nlp,reduce_ents = False)
    
    
    print("NAMES/ORGS IN TXT : ",[pos_tags['PERSON'],pos_tags['ORG']])

    txt = txt.lower()
    
    surs_test = [sur for sur in sh_surs if sur in txt ]
    names_test = [name for name in sh_names if name in txt]
    print("SUR TEST ",surs_test)
    print("NAME TEST ",names_test)
    
    if (len(surs_test)*len(names_test) == 0) and (len(pos_tags['PERSON']) == 0) and (len(pos_tags['ORG']) == 0):
        return "PSC Missing"
    
    pos_tags['PERSON'] = [name for name in pos_tags['PERSON'] if name.lower() not in sh_names]
    to_skip = ['psc','co','ltd','plc','ch','companies house'] 
    pos_tags['ORG'] = [name for name in pos_tags['ORG'] if (name.lower() != comp_name.lower()) and (name.lower() not in sh_names) and (name.lower() not in to_skip)]
    
    print("NAMES/ORGS IN TXT : ",[pos_tags['PERSON'],pos_tags['ORG']])

    to_return = None
    
    if (len(surs_test) == 0) or (len(pos_tags['PERSON']) + len(pos_tags['ORG']) > 0):
        to_return = 'PSC Missing'
        
    check_phrases = ['missing from psc','psc missing','should be psc','should be listed as psc','add psc','should have in psc tab','missing psc','missing as psc',
                    'be listed as psc','be listed as a psc','no psc','missing as a psc','added as psc','should be on psc','noted as a psc','noted on psc','psc is missing',
                    'should have in psc','add as PSC']
    
    phrases_test = [True if phrase in txt else False for phrase in check_phrases]
    
    if any(phrases_test) == True:
        to_return = 'PSC Missing'
    
    if ('missing' in txt or 'should be' in txt) and ('psc' in txt):
        to_return = 'PSC Missing'

    if to_return == 'PSC Missing' and len(pos_tags['PERSON']) + len(pos_tags['ORG']) == 0 and len(names_test) > 0 :
        to_return = 'Missing PSC Found'
        
    return to_return



In [ ]:
def missing_prio(psc_id,df):
    op = check_missing(psc_id,df)
    index = df[df.PSC_DISCREPANCY_ID == psc_id].index[0]

    if op == None:
        op = othercontent(df.loc[index,'CONTACT_DETAIL_CLOB'])
    
    if (op == 'Missing VR') and (df.loc[index,'PSC_DISCREPANCY_OPTION_DESC'] != 'Nature of Control'):
        df.loc[index,'material_nonmaterial'] = 'MATERIAL'
        df.loc[index,'priority'] = 'LOW'
    elif op == 'Missing PSC Found':
        df.loc[index,'material_nonmaterial'] = 'NON-MATERIAL'
        df.loc[index,'priority'] = 'NONE'
    else:
        if df.loc[index,'PSC_DISCREPANCY_OPTION_DESC'] != 'Nature of Control':
            df.loc[index,'material_nonmaterial'] = 'MATERIAL'
            df.loc[index,'priority'] = 'HIGH'
        else:
            op = df.loc[index,'priority_explained']
    
    return op # priority_explained

# Bringing it all together

In [ ]:
%%time
def run_report(df):
    # preprocessing...
    print("Preprocessing") 
    print("BEFORE ACT CODE & DEDUP : ",df.shape)
    df = preprocess_df(df)
    df['act_code_flag'] = df['CONTACT_ID'].apply(lambda x :check_act(x,df_act))

    
    df = df[df['act_code_flag'] != True]
    print("AFTER ACT CODE & DEDUP : ",df.shape)
    
    
    start_inds = list(df.PSC_DISCREPANCY_ID.unique())
    
    #  Types of Discrepancy (Single/Multi-NoNOC/Multi) for whole data set
    optionList = ['Nature of control', 'Date of birth','Name','Company number','Company name'] # Material type of Major Discreapancy (High priority)
    #  List all the discrepancies
    df['Discrepancy'] = df['option_descrip_all'].apply(lambda x: find_noc(x,optionList))
    
    print("DONE\n")
    
    # Extraction and comparison
    df['CHIPS_value'] = None
    df['text_difference'] = None
    df['priority_explained'] = None
    df['priority'] = None
    df['material_nonmaterial'] = None
    
    dfname = df[(df.PSC_DISCREPANCY_OPTION_DESC == 'Name') & (df.Discrepancy == 'Single')]
    dfnoc = df[(df.PSC_DISCREPANCY_OPTION_DESC == 'Nature of control') & (df.Discrepancy == 'Single')]
    dfdob = df[(df.PSC_DISCREPANCY_OPTION_DESC == 'Date of birth') & (df.Discrepancy == 'Single')]
    dfnat = df[(df.PSC_DISCREPANCY_OPTION_DESC == 'Nationality') & (df.Discrepancy == 'Single')]
    dfcname = df[(df.PSC_DISCREPANCY_OPTION_DESC == 'Company name') & (df.Discrepancy == 'Single')]
    
    dfname['text_difference'] = dfname.PSC_DISCREPANCY_ID.apply(lambda x: name_prio(x,dfname))
    print("NAME DONE")
    dfnoc['text_difference'] = dfnoc.PSC_DISCREPANCY_ID.apply(lambda x: noc_prio(x,dfnoc))
    print("NOC DONE")
    dfdob['text_difference'] = dfdob.PSC_DISCREPANCY_ID.apply(lambda x: dob_prio(x,dfdob))
    print("DOB DONE")
    dfnat['text_difference'] = dfnat.PSC_DISCREPANCY_ID.apply(lambda x: nationality_prio(x,dfnat))
    print("NAT DONE")
    
    dfcname['priority'] = 'HIGH'
    dfcname['material_nonmaterial'] = 'MATERIAL'
    dfcname['priority_explained'] = 'MATERIAL PSC TYPE'
    
    
    dfmulti = df[df.Discrepancy != 'Single']
    dfmulti['priority'] = [tag_multiple(x) for x in dfmulti.option_descrip_all.values]
    dfmulti['material_nonmaterial'] = dfmulti['priority'].apply(lambda x: 'MATERIAL' if x != None else 'NON-MATERIAL')
    print("MULTI DONE")
    
    # dealing with PSC Missing
    df_miss = df[(df.PSC_DISCREPANCY_TYPE_DESC == 'PSC Missing') & (df.Discrepancy == 'Single')]
    print("\n================\n",df_miss.shape)
    df_miss['priority_explained'] = df_miss.PSC_DISCREPANCY_ID.apply(lambda x: missing_prio(x,df_miss))
    
    df_rest = other_cat_prio(df[df.Discrepancy == 'Single'])
    print("OTHERS DONE")
    
    frames = [dfname,dfnoc,dfdob,dfnat,dfmulti,dfcname,df_rest,df_miss]
    df_op = pd.concat(frames)
                                                                     
    df_op[(df_op.PSC_DISCREPANCY_OPTION_DESC == 'Nature of control') & (df_op.priority == 'NONE')]['priority_explained'] = df_op[(df_op.PSC_DISCREPANCY_OPTION_DESC == 'Nature of control') & (df_op.priority == 'NONE')].PSC_DISCREPANCY_ID.apply(lambda x: missing_prio(x,df_op))
    
    for ind in df_op[df_op.priority_explained == 'Share & VR'].index:                                                                     
        psc_id = df_op.loc[ind,'PSC_DISCREPANCY_ID']
        try:
            txt_diff = noc_prio(psc_id,df_op)
            df_op.loc[ind,'text_difference'] = txt_diff
        except:
            continue
        
    return df_op

df_op = run_report(df)

In [ ]:
col_sel = ['CREATED_DATE','COMPANY_NUMBER','PSC_DISCREPANCY_ID','CORPORATE_BODY_ID','PSC_DISCREPANCY_TYPE_DESC','PSC_DISCREPANCY_OPTION_DESC','discrepancy_detail','Discrepancy',
                    'option_descrip_all','CHIPS_value','text_difference','priority_explained',
                    'priority','material_nonmaterial']

df_for_team = df_op[col_sel]

In [ ]:
df_for_team = df_for_team.sort_values(by='CREATED_DATE')
#df_for_team.to_csv('FINAL REPORT WEBSERVICES 11.10.csv',index = False)
df_for_team.head()

In [ ]:
df_for_team.to_csv('OUTPUT.csv',index = False)